# runtime.base_net_runtime

> TODO fill in description

In [ ]:
#| default_exp runtime.base_net_runtime

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from abc import ABC, abstractmethod
from types import MappingProxyType
from typing import Type, Tuple, Dict

import fbdev
from fbdev.exceptions import NodeError, EdgeError
from fbdev.comp.packet import Packet
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection, PortID
from fbdev.comp.base_component import BaseComponent
from fbdev.graph.graph_spec import GraphSpec, NodeSpec
from fbdev.graph.packet_registry import TrackedPacket
from fbdev.graph.net import Edge, Node, Net
from fbdev.graph.graph_component import GraphComponentFactory

In [ ]:
#|hide
show_doc(fbdev.runtime.base_net_runtime.BaseNetRuntime)

---

### NetRuntime

>      NetRuntime ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class BaseNetRuntime(ABC):
    @classmethod
    def from_graph(cls, graph:GraphSpec):
        component_type = GraphComponentFactory.create_component(graph)
        net_spec = NodeSpec(component_type)
        net = Net(net_spec)
        return cls(net)
    
    @classmethod
    def execute_graph(cls, graph:GraphSpec, *args, config_vals={}, **kwargs):
        with cls.from_graph(graph) as netrun:
            return netrun.execute(*args, config_vals, **kwargs)
    
    @classmethod
    async def aexecute_graph(cls, graph:GraphSpec, *args, config_vals={}, **kwargs):
        async with cls.from_graph(graph) as netrun:
            return await netrun.aexecute(*args, config=config_vals, **kwargs)
    
    @abstractmethod
    def execute(self, *args, config={}, **kwargs): ...
    
    @abstractmethod
    async def aexecute(self, *args, config_vals={}, **kwargs): ...
    
    @abstractmethod
    async def stop(self): ...
    
    async def __aenter__(self):
        return self

    async def __aexit__(self, exc_type, exc_value, traceback):
        await self.stop()
        
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        asyncio.run(self.stop())